In [1]:
from datasets import load_dataset

# Load the Open-Platypus dataset
dataset = load_dataset("garage-bAInd/Open-Platypus")

/Users/anshujit/work/fine_tuning/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 24926/24926 [00:00<00:00, 700864.26 examples/s]


In [22]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the tokenizer and model
model_path = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
tokenizer  = AutoTokenizer.from_pretrained(model_path)
model  = AutoModelForCausalLM.from_pretrained(model_path)


Loading checkpoint shards: 100%|██████████| 4/4 [00:51<00:00, 12.84s/it]


In [26]:
# Inspect the first example to see the available keys
print(dataset['train'][0])

# Define a function to tokenize the instruction and output
def tokenize_function(example):
    return tokenizer(
        example['instruction'], 
        example['output'],
        padding="max_length", 
        truncation=True
    )

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token' : '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenized_dataset = dataset.map(tokenize_function, batched=True)

{'input': '', 'output': 'To find the probability of the spinner landing on $C$, I need to subtract the probabilities of the spinner landing on $A$ and $B$ from $1$, since the sum of the probabilities of all possible outcomes is $1$. I can write this as an equation: $P(C) = 1 - P(A) - P(B)$. I know that $P(A) = \\frac{1}{3}$ and $P(B) = \\frac{5}{12}$, so I can plug those values into the equation and simplify. I get: $P(C) = 1 - \\frac{1}{3} - \\frac{5}{12} = \\frac{12}{12} - \\frac{4}{12} - \\frac{5}{12} = \\frac{3}{12}$. I can reduce this fraction by dividing the numerator and denominator by $3$, and I get: $P(C) = \\frac{1}{4}$. ', 'instruction': 'A board game spinner is divided into three parts labeled $A$, $B$  and $C$. The probability of the spinner landing on $A$ is $\\frac{1}{3}$ and the probability of the spinner landing on $B$ is $\\frac{5}{12}$.  What is the probability of the spinner landing on $C$? Express your answer as a common fraction.', 'data_source': 'MATH/PRM-800K'}


Map: 100%|██████████| 24926/24926 [13:31<00:00, 30.71 examples/s]


In [27]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'instruction', 'data_source', 'input_ids', 'attention_mask'],
        num_rows: 24926
    })
})

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Define the LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank of the LoRA decomposition
    lora_alpha=32,  # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Target specific modules for adaptation
    lora_dropout=0.05
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [31]:
from transformers import DataCollatorForSeq2Seq, Trainer, TrainingArguments

# Data collator that dynamically pads inputs when batching
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir="./logs",
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    # eval_dataset=tokenized_dataset['train'],  # if available
    data_collator=data_collator
)

# Start training
trainer.train()

RuntimeError: MPS backend out of memory (MPS allocated: 18.00 GB, other allocations: 800.00 KB, max allowed: 18.13 GB). Tried to allocate 224.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).